In [1]:
!pip list


Package                 Version
----------------------- -----------
appnope                 0.1.4
asttokens               3.0.0
attrs                   25.3.0
certifi                 2025.7.14
comm                    0.2.3
debugpy                 1.8.17
decorator               5.2.1
executing               2.2.1
h11                     0.16.0
idna                    3.10
ipykernel               7.1.0
ipython                 9.6.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
jupyter_client          8.6.3
jupyter_core            5.9.1
matplotlib-inline       0.2.1
nest-asyncio            1.6.0
numpy                   2.3.4
outcome                 1.3.0.post0
packaging               25.0
pandas                  2.3.3
parso                   0.8.5
pexpect                 4.9.0
pip                     25.3
platformdirs            4.5.0
prompt_toolkit          3.0.52
psutil                  7.1.2
ptyprocess              0.7.0
pure_eval               0.2.3
Pygments            

In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np

In [4]:
import pandas as pd

In [6]:
# Load the CSV file into a dataframe
df = pd.read_csv('/Users/nagendra_subramanya@optum.com/Library/CloudStorage/OneDrive-Krishna/Nagendra/SelfCode/DatabaseInCSV/DailyLogSummaryForEachDay.csv')

# Display the first few rows of the dataframe
print(df.head())

           Category         ActivityDate  TotalValue
0          SelfHelp  2019-09-26 00:00:00         0.0
1          SelfCode  2019-09-26 00:00:00         0.0
2          SelfTech  2019-09-26 00:00:00         0.0
3          SelfSong  2019-09-26 00:00:00         0.0
4  FitbitDailySteps  2019-09-26 00:00:00     18788.0


In [7]:
# Group by 'Category' and calculate the required statistics
stats_by_category = df.groupby('Category').agg(
    TotalSum=('TotalValue', 'sum'),
    TotalCount=('TotalValue', 'count'),
    Average=('TotalValue', 'mean'),
    Median=('TotalValue', 'median'),
    Mean=('TotalValue', 'mean'),
    StdDev=('TotalValue', 'std')
).reset_index()

# Round the numerical columns to 2 decimal places
stats_by_category[['TotalSum', 'Average', 'Median', 'Mean', 'StdDev']] = stats_by_category[['TotalSum', 'Average', 'Median', 'Mean', 'StdDev']].round(2)

# Calculate zero and non-zero entries grouped by 'Category'
zero_scores = df[df['TotalValue'] == 0].groupby('Category').size().reset_index(name='ZeroScores')
non_zero_scores = df[df['TotalValue'] != 0].groupby('Category').size().reset_index(name='NonZeroScores')

# Merge zero and non-zero scores with the main stats dataframe
stats_by_category = stats_by_category.merge(zero_scores, on='Category', how='left').merge(non_zero_scores, on='Category', how='left')

# Fill NaN values with 0 for ZeroScores and NonZeroScores
stats_by_category[['ZeroScores', 'NonZeroScores']] = stats_by_category[['ZeroScores', 'NonZeroScores']].fillna(0).astype(int)

# Calculate zero and non-zero percentages
stats_by_category['ZeroPercentage'] = ((stats_by_category['ZeroScores'] / stats_by_category['TotalCount']) * 100).round(2)
stats_by_category['NonZeroPercentage'] = ((stats_by_category['NonZeroScores'] / stats_by_category['TotalCount']) * 100).round(2)

# Display the result
print(stats_by_category)



           Category     TotalSum  TotalCount   Average   Median      Mean  \
0  FitbitDailySteps  35477339.00        3441  10310.18  10345.0  10310.18   
1          SelfCode       311.75        2224      0.14      0.0      0.14   
2          SelfHelp      1077.25        2224      0.48      0.5      0.48   
3          SelfSong       115.00        2224      0.05      0.0      0.05   
4          SelfTech       607.00        2224      0.27      0.0      0.27   

    StdDev  ZeroScores  NonZeroScores  ZeroPercentage  NonZeroPercentage  
0  5710.71          85           3356            2.47              97.53  
1     0.36        1708            516           76.80              23.20  
2     0.54         913           1311           41.05              58.95  
3     0.13        1869            355           84.04              15.96  
4     0.45        1387            837           62.37              37.63  
